# Map-Reduce

## Plan

1. **Ask the user** what kind of companies they want to invest in and how many options they want to see.
2. **Find some matching stocks** based on the user’s interests.
3. **In parallel**, gather key information about each stock — like financial reports, market performance, etc. (*This is the “map” step.*)
4. **Analyze all the results** and create a final list that (*This is the “reduce” step.*):
- Describes each stock
- Explains why it’s a good investment
- Ranks them by priority

In [74]:
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI
import operator
from typing import Annotated

class InvestmentAdvisorState(TypedDict):
    financial_area: str
    stock_number: int
    stock_tickers: list[str]
    stock_details: Annotated[list, operator.add]
    recommendation: str

llm = ChatOpenAI(model="gpt-4o-mini")

## Nodes

### Generate list of stocks

In [75]:
from pydantic import BaseModel

class StockTickers(BaseModel):
    stock_tickers: list[str]

def generate_list_of_stocks(state: InvestmentAdvisorState):
    prompt = f"""
    You are an experienced financial analyst. 
    Based on current market trends and public data, suggest the top {state["stock_number"]} publicly traded companies in the area of "{state["financial_area"]}".

    Return only a list of their stock tickers (symbols), without any explanations.
    """
    
    response = llm.with_structured_output(StockTickers).invoke(prompt)
    return {"stock_tickers": response.stock_tickers}

In [ ]:
generate_list_of_stocks({
   "financial_area": "small AI companies",
   "stock_number": 5
})

### Fetch Stock Details (Map Step)

see that we do not have to follow here overall graph state structure!

In [77]:
import yfinance as yf

class StockState(TypedDict):
    ticker: str

def fetch_stock_details(state: StockState):
    period = "1mo"
    stock_symbol = state["ticker"]
    try:
        stock = yf.Ticker(stock_symbol)

        # Retrieve general stock info and historical market data
        stock_info = stock.info  # Basic company and stock data
        stock_history = stock.history(period=period).to_dict()  # Historical OHLCV data

        # Combine both into a single dictionary
        stock = {
            "stock_symbol": stock_symbol,
            "info": stock_info,
            "history": stock_history
        }
        
        return {"stock_details": [str(stock)]}
        
    except Exception as e:
        return {"error": f"Error fetching stock data for {stock_symbol}: {str(e)}"}


In [ ]:
data = fetch_stock_details({
    "ticker": "AI"
})
print(data)

### Analyze all the results (Reduce Step)

In [79]:
from langchain_core.messages import HumanMessage

def generate_stock_recommendations(state: InvestmentAdvisorState):
    financial_data = "\n\n\n".join(state["stock_details"])

    prompt = f"""
    You are a professional investment advisor.

    Below is a list of companies with their financial data:

    {financial_data}

    Your task is to:
    1. Analyze each company based on the provided technical information (consider only companies from the list above).
    2. Write a short paragraph for each company that includes:
    - The company name and ticker
    - A brief description
    - Why it may or may not be a good investment
    3. Rank the companies from best to worst in terms of investment potential.
    4. Return the result as a sorted list (highest priority first), where each item includes:
    - Rank (starting from 1)
    - Ticker
    - Company name
    - Short description
    - Reason for its ranking
    """

    recommendation = llm.invoke([HumanMessage(content=prompt)])

    return {"recommendation": recommendation.content}

### Mapping (Fetch Stock Details)

### Compile

In [ ]:
from IPython.display import Image
from langgraph.graph import END, StateGraph, START

builder = StateGraph(InvestmentAdvisorState)

builder.add_node("generate_list_of_stocks", generate_list_of_stocks)
builder.add_node("fetch_stock_details", fetch_stock_details)
builder.add_node("generate_stock_recommendations", generate_stock_recommendations)


builder.add_edge(START, "generate_list_of_stocks")


# Map - Reducing
from langgraph.constants import Send
def continue_to_details(state: InvestmentAdvisorState):
    return [Send("fetch_stock_details", {"ticker": ticker}) for ticker in state["stock_tickers"]]

builder.add_conditional_edges("generate_list_of_stocks", continue_to_details, ["fetch_stock_details"])



builder.add_edge("fetch_stock_details", "generate_stock_recommendations")
builder.add_edge("generate_stock_recommendations", END)

graph = builder.compile()
Image(graph.get_graph().draw_mermaid_png())

In [ ]:
result = graph.invoke({"financial_area": "small AI companies", "stock_number": 5})
result

In [ ]:
print(result["recommendation"])